In [1]:
# Importing the required libraries
import cv2
import numpy as np
import math

In [2]:
faceprototype = "opencv_face_detector.pbtxt"
face_model = "opencv_face_detector_uint8.pb"

genderprototype = "gender_deploy.prototxt"
gender_model = "gender_net.caffemodel"

ageprototype = "age_deploy.prototxt"
age_model = "age_net.caffemodel"

In [3]:
# Loading the network

face_net = cv2.dnn.readNet(face_model,faceprototype)
gender_net = cv2.dnn.readNet(gender_model,genderprototype)
age_net = cv2.dnn.readNet(age_model,ageprototype)

In [4]:
# Specifying the model mean value

model_mean_value = (78.4263377603, 87.7689143744, 114.895847746)

In [5]:
gender_list = ["Male","Female"]
age_list = ['(0-2)','(4-6)','(8-12)','(15-20)','(25-32)','(38-43)','(48-53)','(60-100)']

**This function mainly makes a frame to detect our face in video**

In [6]:
def frame_face_box(net,frame):
    frWidth = frame.shape[1]
    frHeight = frame.shape[0]
    ar = []
    blob = cv2.dnn.blobFromImage(frame, 1.0, (227,227), [104.0,117.0,123.0], swapRB=False)
    net.setInput(blob)
    detect = net.forward()
    for b in range(detect.shape[2]):
        threshold = detect[0,0,b,2]
        if threshold > 0.7:
            a1 = int(detect[0,0,b,3]*frWidth)
            b1 = int(detect[0,0,b,4]*frHeight)
            a2 = int(detect[0,0,b,5]*frWidth)
            b2 = int(detect[0,0,b,6]*frHeight)
            ar.append([a1,b1,a2,b2])
            cv2.rectangle(frame,(a1,b1),(a2,b2),(0,255,0),1)
    print(detect)
    return frame, ar

In [7]:
padding = 20
#frame[b[1]:b[3],b[0]:b[2]]

In [8]:
def detector(frame):
    frame,box = frame_face_box(face_net,frame)
    for b in box:
        f = frame[max(0,b[1]-padding):min(b[3]+padding, frame.shape[0]-1),max(0,b[0]-padding):min(b[2]+padding, frame.shape[1]-1)]   
        # This is to extract the face from the frame we created
        blob = cv2.dnn.blobFromImage(f, 1.0, (227,227), model_mean_value, swapRB=False)
        gender_net.setInput(blob)
        g_pred = gender_net.forward()
        gender = gender_list[g_pred[0].argmax()]
        age_net.setInput(blob)
        a_pred = age_net.forward()
        age = age_list[a_pred[0].argmax()]
            
        label = "{},{}".format(gender,age)
        cv2.putText(frame, label, (b[0],b[1]-10),cv2.FONT_HERSHEY_PLAIN,0.8, (255,255,255), 1)
    return frame

In [ ]:
image = cv2.imread("man1.jpg")
output = detector(image)
cv2.imshow('image',output)
cv2.waitKey(0)

[[[[0.         1.         0.82272625 ... 0.11680785 0.6949605
    0.6289692 ]
   [0.         1.         0.12716292 ... 3.9716446  5.0718765
    5.2726746 ]
   [0.         1.         0.11947581 ... 3.975919   1.0993047
    5.276057  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]


In [ ]:
video = cv2.VideoCapture(0)
while(video.isOpened()):
    
    while True:
        ret, frame = video.read()
        frame,box = frame_face_box(face_net,frame)
        for b in box:
            f = frame[max(0,b[1]-padding):min(b[3]+padding, frame.shape[0]-1),max(0,b[0]-padding):min(b[2]+padding, frame.shape[1]-1)]     # This is to extract the face from the frame we created
            blob = cv2.dnn.blobFromImage(f, 1.0, (227,227), model_mean_value, swapRB=False)
            gender_net.setInput(blob)
            g_pred = gender_net.forward()
            gender = gender_list[g_pred[0].argmax()]
            age_net.setInput(blob)
            a_pred = age_net.forward()
            age = age_list[a_pred[0].argmax()]
            
            label = "{},{}".format(gender,age)
            cv2.putText(frame, label, (b[0],b[1]-10),cv2.FONT_HERSHEY_PLAIN,0.8, (255,255,255), 1)
        cv2.imshow('img',frame)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    video.release()
    cv2.destroyAllWindows()
    video.stop()

[[[[0.         1.         0.9959921  ... 0.3393248  0.7051931
    0.79298174]
   [0.         1.         0.12638928 ... 3.9697556  5.070694
    5.268247  ]
   [0.         1.         0.1201471  ... 3.9742467  1.0949855
    5.272797  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.996442   ... 0.3368963  0.70715785
    0.79982233]
   [0.         1.         0.12563184 ... 3.970589   5.0712724
    5.2680945 ]
   [0.         1.         0.12047546 ... 3.9740825  1.094492
    5.273379  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9936605  ... 0.33364436 0.707588
    0.7991607 ]
   [0.    

[[[[0.         1.         0.99148834 ... 0.33564526 0.6917591
    0.7702829 ]
   [0.         1.         0.12781297 ... 3.969831   5.0706306
    5.2706122 ]
   [0.         1.         0.1219651  ... 3.9734426  1.095327
    5.2739806 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9929724  ... 0.3302648  0.6940412
    0.7659167 ]
   [0.         1.         0.12823692 ... 3.9694335  5.070997
    5.270103  ]
   [0.         1.         0.12233799 ... 3.97322    1.0956616
    5.2740993 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.992109   ... 0.32722658 0.69126785
    0.76286167]
   [0.   

[[[[0.         1.         0.9998271  ... 0.32496703 0.6510592
    0.75808966]
   [0.         1.         0.12770689 ... 3.9699218  5.0711126
    5.2706842 ]
   [0.         1.         0.11986794 ... 3.9747403  1.095456
    5.273497  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9998068  ... 0.32548255 0.6519729
    0.7607332 ]
   [0.         1.         0.1268977  ... 3.9703245  5.071113
    5.2712145 ]
   [0.         1.         0.11989292 ... 3.975318   1.0946817
    5.273567  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9998267  ... 0.32608032 0.6516472
    0.75625837]
   [0.    

[[[[0.         1.         0.99996257 ... 0.32467833 0.66493076
    0.7332214 ]
   [0.         1.         0.12820101 ... 3.970541   5.071782
    5.271022  ]
   [0.         1.         0.12097643 ... 3.9742875  1.0960144
    5.273955  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9999248  ... 0.32742953 0.6729399
    0.7261512 ]
   [0.         1.         0.12796374 ... 3.9699829  5.0691457
    5.2730703 ]
   [0.         1.         0.12183242 ... 3.9736886  1.0965432
    5.274077  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99977726 ... 0.32515025 0.6812287
    0.70763123]
   [0.  

[[[[0.         1.         0.99973434 ... 0.33338785 0.59067035
    0.68577635]
   [0.         1.         0.12927668 ... 3.968179   5.071835
    5.2707987 ]
   [0.         1.         0.12162603 ... 3.9725375  1.1044766
    5.2722554 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99866986 ... 0.33017766 0.59755456
    0.67999077]
   [0.         1.         0.13300101 ... 0.64276856 0.21783511
    0.7386722 ]
   [0.         1.         0.12830079 ... 3.9687314  5.0716743
    5.271584  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9999721  ... 0.32461202 0.60411286
    0.68040335]
   [0

...

[[[[0.         1.         0.9983559  ... 0.15665986 0.6354499
    0.5928613 ]
   [0.         1.         0.2378214  ... 0.26294738 0.6316352
    0.47053277]
   [0.         1.         0.13033935 ... 0.6399976  0.22344919
    0.7305583 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99997056 ... 0.18226688 0.64622563
    0.5815634 ]
   [0.         1.         0.15206122 ... 0.2798082  0.6355445
    0.4932097 ]
   [0.         1.         0.12884013 ... 3.972803   5.0759053
    5.2679224 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99998593 ... 0.22011198 0.6446895
    0.5880706 ]
   [0.

[[[[0.         1.         0.9996705  ... 0.3021904  0.67322093
    0.7034353 ]
   [0.         1.         0.22008598 ... 0.638735   0.22536895
    0.7323151 ]
   [0.         1.         0.12762052 ... 3.9709206  5.074055
    5.2687016 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.999363   ... 0.30275056 0.678172
    0.7069149 ]
   [0.         1.         0.17261145 ... 0.6419389  0.22074792
    0.730222  ]
   [0.         1.         0.12770474 ... 3.9703145  5.0735154
    5.268803  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99822    ... 0.30335903 0.68326896
    0.70633996]
   [0.

[[[[0.         1.         0.9997303  ... 0.29506987 0.7003978
    0.7131341 ]
   [0.         1.         0.12745056 ... 3.9705882  5.073204
    5.269877  ]
   [0.         1.         0.12203801 ... 3.973869   1.0997522
    5.273573  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9992849  ... 0.29296482 0.70334005
    0.7081208 ]
   [0.         1.         0.12733924 ... 3.970817   5.0734215
    5.2699437 ]
   [0.         1.         0.12260511 ... 3.9727533  1.0997696
    5.2739587 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9994221  ... 0.29025418 0.7000918
    0.7088108 ]
   [0.  

[[[[0.         1.         0.9995266  ... 0.30195037 0.7098286
    0.72485435]
   [0.         1.         0.12713677 ... 3.9703414  5.0731235
    5.2690983 ]
   [0.         1.         0.12195984 ... 3.9732425  1.0996624
    5.2734284 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99951863 ... 0.30206865 0.70964634
    0.72840077]
   [0.         1.         0.12724097 ... 3.9707003  5.073455
    5.2689524 ]
   [0.         1.         0.12206621 ... 3.9738526  1.0992383
    5.273557  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9995259  ... 0.30141377 0.71103764
    0.7314215 ]
   [0. 

[[[[0.         1.         0.9999826  ... 0.3289544  0.7127048
    0.6763032 ]
   [0.         1.         0.12936576 ... 3.9702048  5.074982
    5.266582  ]
   [0.         1.         0.12380621 ... 0.5150858  0.7789401
    0.6817868 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.999985   ... 0.33126864 0.7120752
    0.67591345]
   [0.         1.         0.12916197 ... 3.9697838  5.0744257
    5.2673073 ]
   [0.         1.         0.12359389 ... 3.9713113  1.1039152
    5.272462  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9999789  ... 0.3322017  0.71015745
    0.6764612 ]
   [0.  

In [ ]:
video = cv2.VideoCapture(0)
while(video.isOpened()):
    
    while True:
        ret, frame = video.read()
        frame = detector(frame)
        cv2.imshow('img',frame)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    video.release()
    cv2.destroyAllWindows()
    video.stop()

[[[[0.         1.         0.99993753 ... 0.32960773 0.6902029
    0.70718086]
   [0.         1.         0.1281887  ... 3.9692497  5.07199
    5.270712  ]
   [0.         1.         0.1234979  ... 3.971212   1.1008484
    5.273129  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99992967 ... 0.32003552 0.68552566
    0.6985386 ]
   [0.         1.         0.1273321  ... 3.9704273  5.072778
    5.27054   ]
   [0.         1.         0.12354266 ... 3.9706144  1.1020557
    5.272456  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9999331  ... 0.3214339  0.6859722
    0.70245695]
   [0.    

[[[[0.         1.         0.9999361  ... 0.33023113 0.68585503
    0.7083871 ]
   [0.         1.         0.14139816 ... 0.637381   0.2215702
    0.7304506 ]
   [0.         1.         0.12708147 ... 3.970332   5.0727224
    5.270269  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9998964  ... 0.3308438  0.68628234
    0.70700085]
   [0.         1.         0.16619635 ... 0.6334524  0.21952714
    0.72751176]
   [0.         1.         0.12696302 ... 3.9702315  5.0723495
    5.270889  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99989676 ... 0.3307861  0.6814895
    0.7114967 ]
   [0

[[[[0.         1.         0.99982303 ... 0.31449658 0.6559129
    0.7185784 ]
   [0.         1.         0.12655997 ... 3.9707007  5.0722504
    5.271095  ]
   [0.         1.         0.12534982 ... 0.6351908  0.22495362
    0.73211366]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9998808  ... 0.31688106 0.6570375
    0.7202163 ]
   [0.         1.         0.18924268 ... 0.6384538  0.22229479
    0.73134106]
   [0.         1.         0.12699753 ... 3.9709692  5.072828
    5.2706795 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99995196 ... 0.32059318 0.6599042
    0.7223473 ]
   [0. 

[[[[0.         1.         0.9999752  ... 0.32176673 0.6609289
    0.71700513]
   [0.         1.         0.15200913 ... 0.63137096 0.2254503
    0.73165625]
   [0.         1.         0.12704466 ... 3.9707963  5.0730505
    5.2699294 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9999664  ... 0.32208344 0.6606778
    0.7156186 ]
   [0.         1.         0.13073064 ... 0.63147753 0.22285607
    0.72900265]
   [0.         1.         0.12679178 ... 3.971092   5.0723333
    5.271308  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9999777  ... 0.32297975 0.6590052
    0.71832365]
   [0. 

[[[[0.         1.         0.99997485 ... 0.3235639  0.65936255
    0.7202519 ]
   [0.         1.         0.12701538 ... 3.970846   5.0721173
    5.271324  ]
   [0.         1.         0.12002882 ... 3.9752963  1.098988
    5.2733073 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9999682  ... 0.32373658 0.6597748
    0.71665657]
   [0.         1.         0.18797475 ... 0.6318577  0.2249673
    0.7264448 ]
   [0.         1.         0.12648018 ... 3.9711268  5.07221
    5.271521  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99997437 ... 0.32373917 0.6591892
    0.71842587]
   [0.    

...

[[[[0.         1.         0.999253   ... 0.3471396  0.7437346
    0.7808931 ]
   [0.         1.         0.12326161 ... 3.97167    5.0725465
    5.2668953 ]
   [0.         1.         0.12180715 ... 3.9729273  1.0982853
    5.271969  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99911827 ... 0.3487196  0.7388689
    0.780619  ]
   [0.         1.         0.12429546 ... 3.9705467  5.072124
    5.2668343 ]
   [0.         1.         0.12106913 ... 3.9735456  1.0971003
    5.272755  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99900323 ... 0.34843743 0.7409481
    0.7848228 ]
   [0.   

[[[[0.         1.         0.9974443  ... 0.33390957 0.74141574
    0.77554613]
   [0.         1.         0.12446442 ... 3.9710927  5.0724807
    5.267133  ]
   [0.         1.         0.12114668 ... 3.9735918  1.0984528
    5.2719736 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99552464 ... 0.33310467 0.7376896
    0.7773481 ]
   [0.         1.         0.12484166 ... 3.9702024  5.0716343
    5.2676334 ]
   [0.         1.         0.12139191 ... 3.9733152  1.0980213
    5.2719955 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9940593  ... 0.33074135 0.73373276
    0.7780219 ]
   [0.

[[[[0.         1.         0.99759275 ... 0.33135515 0.7459653
    0.7716343 ]
   [0.         1.         0.12441129 ... 3.9708872  5.07216
    5.26735   ]
   [0.         1.         0.12072847 ... 3.9740915  1.098581
    5.2722425 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99689484 ... 0.33082366 0.7456714
    0.77070236]
   [0.         1.         0.12444656 ... 3.9710238  5.072192
    5.2677417 ]
   [0.         1.         0.12059233 ... 3.9742618  1.0995698
    5.2717204 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9974916  ... 0.33303553 0.7415121
    0.76722294]
   [0.      

[[[[0.         1.         0.9989027  ... 0.33663872 0.74643457
    0.7689645 ]
   [0.         1.         0.124091   ... 3.9714632  5.0730247
    5.2673607 ]
   [0.         1.         0.12100623 ... 3.9742758  1.0984939
    5.272348  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9989361  ... 0.33210593 0.74586344
    0.7668932 ]
   [0.         1.         0.12443871 ... 3.9711535  5.0725923
    5.267744  ]
   [0.         1.         0.12202942 ... 3.9730182  1.098737
    5.272344  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9979765  ... 0.33117956 0.741967
    0.7696169 ]
   [0.  

[[[[0.         1.         0.9945262  ... 0.33637446 0.7326871
    0.77490956]
   [0.         1.         0.131064   ... 0.64121616 0.21906814
    0.7292936 ]
   [0.         1.         0.1263164  ... 3.9700675  5.0714583
    5.269278  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.9971962  ... 0.33418456 0.7326348
    0.7631966 ]
   [0.         1.         0.13252212 ... 0.6438637  0.21855712
    0.7295804 ]
   [0.         1.         0.1269486  ... 3.9701824  5.072267
    5.2681847 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
[[[[0.         1.         0.99845004 ... 0.32819068 0.7368757
    0.75575304]
   [0. 